# Dokumentacja działania implementacji mikroserwisu

Stworzone endpointy w ramach serwisu:

![image](https://user-images.githubusercontent.com/59453698/148667526-79363af5-8b04-4008-b114-5d286746e965.png)


In [1]:
import random

import requests

## Możliwość zmiany modelu, który dostarcza predykcje
Dostępne modele:
- Las losowy
- Sieć neuronowa
- Model naiwny

In [2]:
# Zmiana na model lasu losowego
requests.get(url="http://localhost:8000/model/choose_model/random_forest").json()

{'result': 'Model changed to random_forest'}

In [3]:
# Zmiana na model sieci neuronowej
requests.get(url="http://localhost:8000/model/choose_model/nn").json()

{'result': 'Model changed to nn'}

In [4]:
# Zmiana na model naiwny
requests.get(url="http://localhost:8000/model/choose_model/naive").json()

{'result': 'Model changed to naive'}

## Możliwość wyświetlenia obecnie wybranego modelu

In [5]:
requests.get(url="http://localhost:8000/model/current_model").json()

{'model_name': 'naive'}

## Możliwość serwowania predykcji
Wcześniej należy wysłać requesta wybierającego porządany przez użytkownika model, który ma dostarczać predykcje

##### Predykcja modelem lasu losowego

In [6]:
requests.get(url="http://localhost:8000/model/choose_model/random_forest").json()

{'result': 'Model changed to random_forest'}

In [7]:
requests.post(url="http://localhost:8000/model/predict", json={"data": [5.0, 19.0,1000.98,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0,7.0,28.0,15.0,2.0]}).json()

{'prediction:': 0.0}

##### Predykcja modelem sieci neuronowej

In [8]:
requests.get(url="http://localhost:8000/model/choose_model/nn").json()

{'result': 'Model changed to nn'}

In [9]:
requests.post(url="http://localhost:8000/model/predict", json={"data": [5.0, 19.0,1000.98,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0,7.0,28.0,15.0,2.0]}).json()

{'prediction:': 1.0}

##### Predykcja modelem naiwnym

In [10]:
requests.get(url="http://localhost:8000/model/choose_model/naive").json()

{'result': 'Model changed to naive'}

In [11]:
requests.post(url="http://localhost:8000/model/predict", json={"data": [5.0, 19.0,1000.98,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0,7.0,28.0,15.0,2.0]}).json()

{'prediction:': 0.0}

## Możliwość realizacji eksperymentu A/B
W przypadku tego projektu, eksperyment jest realizowany między modelem A - lasu losowego, a modelem B - naiwnym (wykonującym predykcję na podstawie czasu trwania sesji). Miarę, jaką zastosujemy to, zgodnie z założeniami wstępnymi, accuracy dla obu modeli.

Aby rozpocząć na nowo eksperyment A/B, użytkownik przesyła do mikroserwisu request'a, restartującego zbiór próbek, który mikroserwis przygotuje, aby móc przeprowadzić eksperyment.

In [12]:
requests.post(url="http://localhost:8000/model/ab/reset_ab").json()

{'msg': 'Reset of AB predictions file performed!'}

Następnie rozpoczyna się etap zbierania próby do przeprowadzenia eksperymentu A/B. W naszym przypadku założyliśmy, że do mikroserwisu przekazywane są dane wejściowe dla modelów wraz z informacją, jakiej sesji one dotyczą.

Dla ułatwienia zadania stworzyliśmy funkcję, która dla podanego session_id ze zbioru testowego, zwraca ciało zapytania, które jest wysyłane do mikroserwisu.

Podział na próby jest wykonywany w taki sposób, że sesje o numerze nieparzystym trafiają do grupy z modelem naiwnym, natomiast te z parzystym - do grupy z modelem zaawansowanym. Wyniki są zapisywane do pliku w postaci logów, które potem mikroserwis wykorzysta aby porównać modele.

In [17]:
from tests.microservice_test import create_input_for_microservice
import pandas as pd

test_df = pd.read_json(path_or_buf='../data/processed/test_set.jsonl')
possible_sessions_id = test_df['session_id'].values.tolist()
responses = []
SAMPLE_SIZE = 150

for index in range(SAMPLE_SIZE):
    inp_data = create_input_for_microservice(possible_sessions_id[index])
    resp = requests.post(url="http://localhost:8000/model/ab/add_sample", json=inp_data).json()
    responses.append(resp)

responses = random.sample(responses, 10)

for el in responses:
    print(el)

C:\Users\01149762\Documents\AI\purchase-in-session-analysis\venv\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


{'model_name': 'naive', 'prediction': 1}
{'model_name': 'naive', 'prediction': 1}
{'model_name': 'naive', 'prediction': 1}
{'model_name': 'random_forest', 'prediction': 0.0}
{'model_name': 'naive', 'prediction': 0}
{'model_name': 'random_forest', 'prediction': 0.0}
{'model_name': 'random_forest', 'prediction': 0.0}
{'model_name': 'random_forest', 'prediction': 0.0}
{'model_name': 'random_forest', 'prediction': 1.0}
{'model_name': 'random_forest', 'prediction': 0.0}


Przykładowy wygląd loga, który tworzy mikroserwis, zbierając dane do przeprowadzenia eksperymentu A/B:

![image](https://user-images.githubusercontent.com/59453698/148668439-c9b70190-f10c-4541-977b-a203812be4ec.png)


Na końcu, gdy już uzbierzemy próbę o wystarczającej wielkości, możemy wykonać porównanie modeli w kontekście tych zebranych prób.

<i>W tym projekcie założyliśmy, że próbki, które tworzą próbę w eksperymencie a/b pochodzą ze zbioru testowego - tzn. mikroserwis, gdy już uzbiera próbę o wystarczającej wielkości, porównuje oba modele zestawiając swój log (przechowujący predykcje modeli w ramach konkretnych sesji) ze zbiorem testowym (porównuje konkretne session_id).</i>

In [18]:
requests.post(url="http://localhost:8000/model/ab/perform_ab").json()

{'Naive accuracy on A/B test:': '68.116%',
 'Random forest accuracy on A/B test:': '100.000%'}